In [24]:
def bin8(num):
    if(num>255):
        print("num is",num)
        raise Exception("num >255!")
    outnum = bin(num)[2:]
    while(len(outnum)!=8):
        outnum = '0' + outnum
    return outnum

def one_element_coding(num):
    outnum = '0' * num + '1'
    return outnum

def golomb_coding(MErrval,k,LIMIT=32,qbpp=8):
    MErrval_R_k = MErrval>>k
    outstr = ''
    if MErrval_R_k < LIMIT-qbpp-1:
        outstr+=one_element_coding(MErrval_R_k)
        outstr+=bin8(MErrval)[-k:]
    else:
        outstr+=one_element_coding(LIMIT-qbpp-1)
        outstr+=bin8(MErrval-1)[-qbpp:]
    return outstr

def golomb_decoding(k,LIMIT=32,qbpp=8):
    global bitstream,bitstream_cnt
    subtraction = 0
    while bitstream[bitstream_cnt] == '0':
        subtraction += 1
        bitstream_cnt += 1
    bitstream_cnt += 1
    if subtraction < LIMIT - qbpp - 1:
        remainder = int(bitstream[bitstream_cnt:bitstream_cnt+k],2)
        quotient = (subtraction << k) + remainder
        value = quotient
    else:
        remainder_bits = bitstream[bitstream_cnt:bitstream_cnt+qbpp]
        quotient = int(remainder_bits, 2) + 1
        value = quotient
    return value

def Golomb_k(Q):
    global A,B,C,N
    k=0
    while((N[Q]<<k)<A[Q] and k<=7):
        k+=1
    return k

# def error_mapping(Errval,Q,k):
#     if(k==0 and 2*B[Q]<=-N[Q]):# special mapping
#         if(Errval >= 0):
#             MErrval = 2*Errval+1
#         else:
#             MErrval = -2*(Errval+1)
#     else:# regular mapping
#         if Errval>=0:
#             MErrval = 2*Errval
#         else:
#             MErrval = -2*Errval-1
#     # print(Errval,MErrval)
#     return MErrval

def error_mapping_revise(MErrval,Q,k):
    if(k==0 and 2*B[Q]<=-N[Q]):
        if MErrval%2 == 0:
            Errval = -MErrval/2-1
        else:
            Errval = (MErrval-1)/2
    else:
        if MErrval%2 == 0:
            Errval = MErrval/2
        else:
            Errval = -(MErrval+1)/2
    return Errval
def get_next_ij(i,j):
    global pic_origin,pixel_cnt
    width = len(pic_origin[0])
    height = len(pic_origin)
    pixel_cnt = i*width + j
    pixel_cnt = pixel_cnt + 1
    new_i = pixel_cnt//width
    new_j = pixel_cnt%height
    return new_i,new_j


In [25]:
def init_pic_expand(width,height):
    pic_expand = [[0 for _ in range(width+2)] for _ in range(height+1)]
    return pic_expand
def insert_pixel(i,j,pixel):
    global pic_expand
    pic_expand[i+1][j+1] = pixel

def pic_shrink(pic_expand):
    width = len(pic_expand[0])-2
    height = len(pic_expand)-1
    
    pic_origin = [[0 for _ in range(width)] for _ in range(height)]

    for i in range(width):
        for j in range(height):
            pic_origin[i][j] = pic_expand[i+1][j+1]
    return pic_origin
def get_pixels(expand_pic,hang_index,lie_index):
    _hang_index = hang_index+1
    _lie_index = lie_index+1

    # x = expand_pic[_hang_index     ][_lie_index      ]
    a = expand_pic[_hang_index     ][_lie_index    - 1  ]
    b = expand_pic[_hang_index  -1 ][_lie_index    ]
    c = expand_pic[_hang_index - 1 ][_lie_index   -1 ]
    d = expand_pic[_hang_index -1  ][_lie_index  +1  ]
    return a,b,c,d
def get_gradient(a,b,c,d):
    D1 = d-b
    D2 = b-c
    D3 = c-a
    return D1,D2,D3

def mode_choose(D1,D2,D3):
    '''
    return mode = 0 : run mode
    return mode = 1 : regular mode
    '''
    if D1==0 and D2==0 and D3==0:
        mode = 0
    else:
        mode = 1
    return mode

def quantinize(D):
    if D<=-21:
        Q=-4
    elif D<=-7:
        Q=-3
    elif D<=-3:
        Q=-2
    elif D<0:
        Q=-1
    elif D==0:
        Q=0
    elif D<=3:
        Q=1
    elif D<=7:
        Q=2
    elif D<=21:
        Q=3
    else:
        Q=4
    return Q

def mapping_and_sign(Q1,Q2,Q3):
    if Q1<0 or (Q1==0 and Q2<0) or (Q1==0 and Q2==0 and Q3<0):
        SIGN = -1
        Q1,Q2,Q3=-Q1,-Q2,-Q3
    else:
        SIGN = 1
    Q = 81*Q1+9*Q2+Q3
    return Q,SIGN

def prediction(a,b,c):
    if c>max(a,b):
        Px = min(a,b)
    elif c<min(a,b):
        Px = max(a,b)
    else:
        Px = a+b-c
    return Px

def prediction_corrected(Px,SIGN,Q,MAXVAL=255):
    global C
    Px = Px + SIGN*C[Q]
    if(Px>MAXVAL):
        Px = MAXVAL
    if(Px<0):
        Px = 0
    return Px
def update_params(Q,Errval):
    global A,B,C,N,RESET,MAX_C,MIN_C
    B[Q] = B[Q] + Errval
    A[Q] = (A[Q] + Errval) if (Errval>=0) else (A[Q] - Errval)
    if N[Q]==RESET:
        A[Q] = A[Q] >> 1
        if B[Q] >= 0:
            B[Q] = B[Q] >> 1
        else:
            B[Q] = -((1-B[Q])>>1)
        N[Q]>>1
    N[Q]=N[Q]+1

    if B[Q]<=-N[Q]:
        B[Q]=B[Q]+N[Q]
        if C[Q]>MIN_C:
            C[Q] = C[Q] - 1
        if B[Q]<=-N[Q]:
            B[Q] = -N[Q] + 1
    elif B[Q]>0:
        B[Q] = B[Q]-N[Q]
        if C[Q] < MAX_C:
            C[Q] = C[Q]+1
        if B[Q]>0:
            B[Q]=0

In [26]:
def regular_decode(a,b,c,d):
    D1,D2,D3 = get_gradient(a,b,c,d)
    Q,SIGN = mapping_and_sign(quantinize(D1),quantinize(D2),quantinize(D3))
    Px = prediction(a,b,c)
    print(Px)
    Px_corrected = prediction_corrected(Px,SIGN,Q)
    k = Golomb_k(Q)
    print(k)
    MErrval = golomb_decoding(k)
    print(MErrval)
    Errval = error_mapping_revise(MErrval,Q,k)
    print(Errval)
    x = Errval/SIGN + Px_corrected
    update_params(Q,Errval)
    return x

def run_decode(a,b,c,d):
    global bitstream,bitstream_cnt,J,RUNindex,i,j
    current_bit = bitstream[bitstream_cnt]

    if current_bit == '1':
        fill_num = 0
        while fill_num<=(1<<J[RUNindex]) and j!=len(pic_origin[0])-1:
            insert_pixel(i,j,a)
            i,j = get_next_ij(i,j)
            fill_num += 1
        if fill_num == 1<<J[RUNindex] and RUNindex<31:
            RUNindex+=1
    else:#current_bit == '0'
        fill_dest = 0 if bitstream[bitstream_cnt:bitstream_cnt+J[RUNindex]] == '' else int(bitstream[bitstream_cnt:bitstream_cnt+J[RUNindex]],2)
        fill_num = 0
        while fill_num<=fill_dest:
            insert_pixel(i,j,a)
            i,j = get_next_ij(i,j)
            fill_num += 1
        if RUNindex>0:
            RUNindex-=1
        run_decode_inter(a,b,c,d)
        

def ModRange_runmode(Errval,RANGE=256):
    ### 两个条件均自行修改过
    if Errval<(-RANGE/2):
        Errval+=RANGE
    elif Errval>=((RANGE-1)/2):
        Errval-=RANGE
    return Errval


def Errval_SIGN(RItype,a,b,Errval):
    if RItype == 0 and a>b:
        Errval = -Errval
        SIGN = -1
    else:
        SIGN = 1
    Errval = ModRange_runmode(Errval)
    return Errval,SIGN
def Errval_demap(EMErrval,RItype):
    if (EMErrval+RItype)%2==0:
        Errval = (EMErrval+RItype)//2
    else:
        Errval = -(EMErrval+RItype+1)//2
    return Errval

def run_decode_inter(a,b,c,d):
    global A,B,C,N,Nn,LIMIT
    if a==b:
        RItype = 1
        Px = a
        TEMP = A[366] + (N[366]>>1)
    else:
        RItype = 0
        Px = b
        TEMP = A[365]

    Q = RItype + 365
    k=0
    while((N[Q]<<k)<TEMP and k<=7):
        k+=1
    EMErrval =  golomb_decoding(k,LIMIT=LIMIT-J[RUNindex]-1)
    Errval = Errval_demap(EMErrval,RItype)
    x = Px + Errval
    insert_pixel(i,j,x)
    if Errval<0:
        Nn[Q]+=1
    A[Q]+=(EMErrval+1-RItype)>>1
    if N[Q]==RESET:
        A[Q] = A[Q]<<1
        N[Q] = N[Q]<<1
        Nn[Q] = Nn[Q]<<1
    N[Q]+=1

    # Errval = x - Px
    # Errval,SIGN = Errval_SIGN(RItype,a,b,Errval)

        

In [27]:
if __name__ == "__main__":

    # for 8 bits picture
    MAXVAL=255
    RANGE=256
    bpp=8
    qbpp=8
    LIMIT=32
    MAX_C=127
    MIN_C=-128
    RESET = 64
    MAX_C = 127
    MIN_C = -128

    # A，N从0到366，B，C从0到364，Nn从365到366，A初始化为全4，其余初始化都为全0
    A = [4 for _ in range(367)]
    B = [0 for _ in range(365)]
    C = [0 for _ in range(365)]
    N = [1 for _ in range(367)]
    Nn= {365:0,366:0}

    bitstream = '1100000000000000000000000110110010000000001000001000111000000001110000000000000000000000010101110100000000000000000000000110111011100110000000000000000000000001101111000001100000000000000000000000010111011000000000000000000010010001011'
    # bitstream = '00000000000000000000000110110010'
    RUNcnt = 0
    RUNindex = 0
    RUNval = 0
    J = [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    bitstream_cnt=0
    pic_expand = init_pic_expand(4,4)
    i=0
    j=0
    Pixel_cnt = 0
    while(bitstream_cnt < len(bitstream)):
        a,b,c,d = get_pixels(pic_expand,i,j)
        D1,D2,D3 = get_gradient(a,b,c,d)
        mode = mode_choose(D1,D2,D3)
        if (mode == 1):# regular mode
            insert_pixel(i,j,regular_decode(a,b,c,d))
            i,j = get_next_ij(i,j)
        else:# run mode
            run_decode(a,b,c,d)
    
    pic_origin = pic_shrink(pic_expand)
    print(pic_origin)

: 

: 